# CoDaS-HEP Columnar Data Analysis, part 2

This is the second of three sessions on [columnar data analysis](https://indico.cern.ch/event/1151367/timetable/#41-columnar-data-analysis), presented at CoDaS-HEP at 12:30pm on August 3, 2022 by Jim Pivarski and Ioana Ifrim.

See the [GitHub repo](https://github.com/jpivarski-talks/2022-08-03-codas-hep-columnar-tutorial#readme) for instructions on how to run it.

## From ROOT files into arrays

<br><br><br><br><br>

### Uproot

<br><br><br><br><br>

### Awkward Array

<br><br><br><br><br>

## Project: H → ZZ → 4ℓ

<br><br><br><br><br>

### 4 leptons of the same flavor

<br><br><br><br><br>

### Opposite charges

<br><br><br><br><br>

### On your own: the H → ZZ → 2μ2e case

<br><br><br><br><br>

### Hint!